In [1]:
import markdown
from jsmin import jsmin
import ast
import re
import os
from csscompressor import compress

In [4]:
# read markdown article
with open('article.md', encoding='utf-8') as text:
    parts = text.read().split('---')
    header = parts[0]
    main = parts[1]

In [31]:
art

{'authors': 'Jan Cibulka a Petr Kočí',
 'coverimg': 'https://interaktivni.rozhlas.cz/brexit/media/cover.jpg',
 'coverimg_note': "Foto <a href='#'>ČTK</a>",
 'libraries': '<script src="https://unpkg.com/jquery@3.2.1"></script>\n',
 'perex': 'Praha se denně nafoukne o polovinu, v centru jsou návštěvníci v převaze',
 'styles': '',
 'title': 'Data z mobilů:'}

In [24]:
# article base
art = {}
for row in header.split('\n'):
    var = row.split(':')[0]
    val = ':'.join(row.split(':')[1:]).lstrip()
    if len(val) > 0:
        if val.startswith('['):
            art[var] =  ast.literal_eval(val)
        else:
            art[var] =  val

In [26]:
#names of authors
tmp = ', '.join(art['authors'])
art['authors'] = ' a '.join(tmp.rsplit(', ', 1))

In [28]:
# format external JS links
tmp = ''
for lib in art['libraries']:
    tmp += '<script src="{0}"></script>\n'.format(lib)
art['libraries'] = tmp

In [30]:
# format external CSS links 
tmp = ''
for styl in art['styles']:
    tmp += '<link rel="stylesheet" type="text/css" href="{0}">\n'.format(styl)
art['styles'] = tmp

In [32]:
# article content
art['content'] = markdown.markdown(main)

In [33]:
#read snowfall template
with open('./templates/snowfall.html', encoding='utf-8') as t:
    template = t.read()

In [34]:
# fill template
for variable in re.findall(r"\{(\w+)\}", template):
    template = template.replace('{' + variable + '}', str(art[variable]))

In [35]:
# pack JSscripts
temp = ''
for script in os.listdir('./js/'):
    with open('./js/' + script, encoding='utf-8') as js_file:
        jmin = jsmin(js_file.read())
        temp += jmin

template = template + '<script>' + temp + '</script>\n' 

In [36]:
# pack styles
temp = ''
for script in os.listdir('./styles/'):
    with open('./styles/' + script, encoding='utf-8') as css_file:
        csmin = compress(css_file.read())
        temp += csmin

template = '<style>' + temp + '</style>\n' + template

In [37]:
#write template
with open('./article.html', 'w', encoding='utf-8') as f:
    f.write(template)